In [5]:
import sys
import time
import config
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extrair_inteiro(texto):
	try:
		i = texto.rindex(' ')
		sem_unidade = texto[:i]

		# Às vezes, esse valor pode iniciar pelo ano...
		i = sem_unidade.find(' ')
		if i >= 0:
			sem_unidade = sem_unidade[(i + 1):]

		sem_virgula = sem_unidade.replace(',', '')

		return int(sem_virgula)
	except:
		return 0

driver = webdriver.Chrome()
driver.get(config.url_inicial)

The chromedriver version (141.0.7390.54) detected in PATH at C:\SeleniumDrivers\chromedriver.exe might not be compatible with the detected chrome version (142.0.7444.176); currently, chromedriver 142.0.7444.175 is recommended for chrome 142.*, so it is advised to delete the driver in PATH and retry


In [6]:
links = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ipc-title > a.ipc-title-link-wrapper'))
)

filmes = []

for i in range(0, 20):
	filmes.append({
		"url": links[i].get_attribute("href"),
		"titulo": "",
		"duracao": "",
		"ano": 0,
		"classificacao_etaria": "",
		"nota": 0,
		"posicao": i + 1,
		"generos": [],
		"atores": [],
	})
	

In [7]:
for contador_filme in range(20):
	filme = filmes[contador_filme]
	driver.get(filme["url"])

	time.sleep(1)

	body = driver.find_element(By.TAG_NAME, 'body')

	for i in range(15):
		body.send_keys(Keys.PAGE_DOWN)
		time.sleep(2)

	titulo = WebDriverWait(driver, 1).until(
		EC.presence_of_element_located((By.CSS_SELECTOR, 'span.hero__primary-text'))
	)

	filme["titulo"] = titulo.text.strip()

	meta = WebDriverWait(driver, 1).until(
		EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.sc-af040695-0.iOwuHP > ul > li'))
	)

	if len(meta) > 0:
		filme["ano"] = int(meta[0].text.strip())
		if len(meta) > 1:
			filme["classificacao_etaria"] = meta[1].text.strip()
			if len(meta) > 2:
				filme["duracao"] = meta[2].text.strip()

	try:
		nota = WebDriverWait(driver, 5).until(
			EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-4dc495c1-1.lbQcRY'))
		)

		filme["nota"] = float(nota[0].text.strip().replace(",", "."))
	except:
		filme["nota"] = 0

	for i in range(3):
		try:
			generos = WebDriverWait(driver, 5).until(
				EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li[data-testid="storyline-genres"] div a'))
			)
			break
		except:
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)
			body.send_keys(Keys.PAGE_UP)
			time.sleep(0.5)

	filme["generos"] = []
	filme["atores"] = []

	for genero in generos:
		filme["generos"].append(genero.text.strip())

	atores = WebDriverWait(driver, 1).until(
		EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ipc-shoveler.title-cast__grid a[data-testid="title-cast-item__actor"]'))
	)

	for ator in atores:
		filme["atores"].append(ator.text.strip())

In [23]:
filmes

[{'url': 'https://www.imdb.com/pt/title/tt26443597/?ref_=chtmvm_t_1',
  'titulo': 'Zootopia 2',
  'duracao': '1 h 48 min',
  'ano': 2025,
  'classificacao_etaria': 'Livre',
  'nota': 7.7,
  'posicao': 1,
  'generos': ['Animação',
   'Ação',
   'Aventura',
   'Comédia',
   'Policial',
   'Família',
   'Mistério'],
  'atores': ['Ginnifer Goodwin',
   'Jason Bateman',
   'Ke Huy Quan',
   'Fortune Feimster',
   'Andy Samberg',
   'David Strathairn',
   'Idris Elba',
   'Shakira',
   'Patrick Warburton',
   'Quinta Brunson',
   'Danny Trejo',
   'Nate Torrence',
   'Bonnie Hunt',
   'Don Lake',
   'Michelle Gomez',
   'David Fane',
   "Joe Anoa'i",
   'CM Punk']},
 {'url': 'https://www.imdb.com/pt/title/tt1312221/?ref_=chtmvm_t_2',
  'titulo': 'Frankenstein',
  'duracao': '2 h 29 min',
  'ano': 2025,
  'classificacao_etaria': '18',
  'nota': 7.5,
  'posicao': 2,
  'generos': ['Drama', 'Fantasia', 'Terror', 'Ficção científica'],
  'atores': ['Oscar Isaac',
   'Jacob Elordi',
   'Christoph W

In [10]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
from config import conexao_banco

engine = create_engine(conexao_banco)

In [11]:
from sqlalchemy.exc import SQLAlchemyError

def salvar_filme(filmes):
    ids_filmes = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            try:
                titulo = filme.get("titulo", "").strip()
                if not titulo:
                    print("ERRO: Filme sem título. Ignorado.")
                    continue

                existente = sessao.execute(
                    text("SELECT id FROM filme WHERE titulo = :titulo"),
                    {"titulo": titulo}
                ).fetchone()

                if existente:
                    ids_filmes[titulo] = existente[0]
                    continue

                sessao.execute(
                    text("""
                        INSERT INTO filme (titulo, duracao, ano, classificacao_etaria)
                        VALUES (:titulo, :duracao, :ano, :classificacao_etaria)
                    """),
                    {
                        "titulo": titulo,
                        "duracao": filme.get("duracao"),
                        "ano": filme.get("ano"),
                        "classificacao_etaria": filme.get("classificacao_etaria")
                    }
                )

                novo_id = sessao.execute(text("SELECT LAST_INSERT_ID()")).fetchone()[0]
                ids_filmes[titulo] = novo_id

            except SQLAlchemyError as e:
                print(f"[FILME] ERRO ao inserir '{titulo}': {e}")
                sessao.rollback()

    return ids_filmes


def salvar_ator(filmes):
    ids_atores = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme.get("atores", []):
                try:
                    nome = nome.strip()
                    if not nome:
                        continue

                    if nome in ids_atores:
                        continue

                    existente = sessao.execute(
                        text("SELECT id FROM ator WHERE nome = :nome"),
                        {"nome": nome}
                    ).fetchone()

                    if existente:
                        ids_atores[nome] = existente[0]
                        continue

                    sessao.execute(
                        text("INSERT INTO ator (nome) VALUES (:nome)"),
                        {"nome": nome}
                    )

                    novo_id = sessao.execute(text("SELECT LAST_INSERT_ID()")).fetchone()[0]
                    ids_atores[nome] = novo_id

                except SQLAlchemyError as e:
                    print(f"[ATOR] ERRO ao inserir '{nome}': {e}")
                    sessao.rollback()

    return ids_atores


def salvar_genero(filmes):
    ids_generos = {}

    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            for nome in filme.get("generos", []):
                try:
                    nome = nome.strip()
                    if not nome:
                        continue

                    if nome in ids_generos:
                        continue

                    existente = sessao.execute(
                        text("SELECT id FROM genero WHERE nome = :nome"),
                        {"nome": nome}
                    ).fetchone()

                    if existente:
                        ids_generos[nome] = existente[0]
                        continue

                    sessao.execute(
                        text("INSERT INTO genero (nome) VALUES (:nome)"),
                        {"nome": nome}
                    )

                    novo_id = sessao.execute(text("SELECT LAST_INSERT_ID()")).fetchone()[0]
                    ids_generos[nome] = novo_id

                except SQLAlchemyError as e:
                    print(f"[GENERO] ERRO ao inserir '{nome}': {e}")
                    sessao.rollback()

    return ids_generos


def salvar_ranking(filmes, ids_filmes):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            try:
                titulo = filme["titulo"]
                id_filme = ids_filmes[titulo]

                existente = sessao.execute(
                    text("""
                        SELECT 1 FROM ranking
                        WHERE id_filme = :id_filme AND data = CURDATE()
                    """),
                    {"id_filme": id_filme}
                ).fetchone()

                if existente:
                    continue

                sessao.execute(
                    text("""
                        INSERT INTO ranking (id_filme, nota, posicao, data)
                        VALUES (:id_filme, :nota, :posicao, CURDATE())
                    """),
                    {
                        "id_filme": id_filme,
                        "nota": filme.get("nota"),
                        "posicao": filme.get("posicao")
                    }
                )

            except SQLAlchemyError as e:
                print(f"[RANKING] ERRO ao inserir ranking de '{titulo}': {e}")
                sessao.rollback()


def salvar_genero_filme(filmes, ids_filmes, ids_generos):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            id_filme = ids_filmes[filme["titulo"]]
            for nome_genero in filme.get("generos", []):
                try:
                    nome_genero = nome_genero.strip()
                    id_genero = ids_generos[nome_genero]

                    existente = sessao.execute(
                        text("""
                            SELECT 1 FROM genero_filme
                            WHERE id_filme = :id_filme AND id_genero = :id_genero
                        """),
                        {"id_filme": id_filme, "id_genero": id_genero}
                    ).fetchone()

                    if existente:
                        continue

                    sessao.execute(
                        text("""
                            INSERT INTO genero_filme (id_filme, id_genero)
                            VALUES (:id_filme, :id_genero)
                        """),
                        {"id_filme": id_filme, "id_genero": id_genero}
                    )

                except SQLAlchemyError as e:
                    print(f"[GENERO_FILME] ERRO para {filme['titulo']} - {nome_genero}: {e}")
                    sessao.rollback()


def salvar_elenco(filmes, ids_filmes, ids_atores):
    with Session(engine) as sessao, sessao.begin():
        for filme in filmes:
            id_filme = ids_filmes[filme["titulo"]]
            for nome_ator in filme.get("atores", []):
                try:
                    nome_ator = nome_ator.strip()
                    id_ator = ids_atores[nome_ator]

                    existente = sessao.execute(
                        text("""
                            SELECT 1 FROM elenco
                            WHERE id_filme = :id_filme AND id_ator = :id_ator
                        """),
                        {"id_filme": id_filme, "id_ator": id_ator}
                    ).fetchone()

                    if existente:
                        continue

                    sessao.execute(
                        text("""
                            INSERT INTO elenco (id_ator, id_filme)
                            VALUES (:id_ator, :id_filme)
                        """),
                        {"id_ator": id_ator, "id_filme": id_filme}
                    )

                except SQLAlchemyError as e:
                    print(f"[ELENCO] ERRO para {filme['titulo']} - {nome_ator}: {e}")
                    sessao.rollback()


In [12]:
ids_filmes = salvar_filme(filmes)
ids_atores = salvar_ator(filmes)
ids_generos = salvar_genero(filmes)

salvar_elenco(filmes, ids_filmes, ids_atores)
salvar_genero_filme(filmes, ids_filmes, ids_generos)
salvar_ranking(filmes, ids_filmes)


In [10]:
print(filmes)

[{'url': 'https://www.imdb.com/pt/title/tt1312221/?ref_=chtmvm_t_1', 'titulo': 'Frankenstein', 'duracao': '2 h 29 min', 'ano': 2025, 'classificacao_etaria': '18', 'nota': 7.5, 'posicao': 1, 'generos': ['Drama', 'Fantasia', 'Terror', 'Ficção científica'], 'atores': ['Oscar Isaac', 'Jacob Elordi', 'Christoph Waltz', 'Mia Goth', 'Felix Kammerer', 'Charles Dance', 'David Bradley', 'Lars Mikkelsen', 'Christian Convery', 'Nikolaj Lie Kaas', 'Kyle Gatehouse', 'Lauren Collins', 'Sofia Galasso', 'Joachim Fjelstrup', 'Ralph Ineson', 'Peter Millard', 'Peter MacNeill', 'Burn Gorman']}, {'url': 'https://www.imdb.com/pt/title/tt19847976/?ref_=chtmvm_t_2', 'titulo': 'Wicked: Parte II', 'duracao': '2 h 17 min', 'ano': 2025, 'classificacao_etaria': '10', 'nota': 7.1, 'posicao': 2, 'generos': ['Família', 'Fantasia', 'Musical', 'Romance'], 'atores': ['Cynthia Erivo', 'Ariana Grande', 'Jeff Goldblum', 'Michelle Yeoh', 'Jonathan Bailey', 'Ethan Slater', 'Marissa Bode', 'Colman Domingo', 'Bowen Yang', 'Bron

In [ ]:
import pandas as pd
import random
from datetime import date, timedelta
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
from config import conexao_banco

engine = create_engine(conexao_banco)
# -----------------------------------------------------
# BUSCA O RANKING DO PRIMEIRO DIA
# -----------------------------------------------------
query = """
SELECT id_filme, nota, posicao, data
FROM ranking
WHERE data = '2025-11-01'
ORDER BY posicao
LIMIT 20;
"""

df = pd.read_sql(query, engine)

# -----------------------------------------------------
# FUNÇÃO DE SIMULAÇÃO
# -----------------------------------------------------
def simular_variacao(dia_anterior):
    novo = dia_anterior.copy()

    # varia nota moderadamente
    novo["nota"] = novo["nota"].apply(
        lambda x: round(max(0, min(10, x + random.uniform(-0.2, 0.3))), 1)
    )

    # varia posição moderadamente
    novas_posicoes = []
    for p in novo["posicao"]:
        if random.random() < 0.33:
            p = max(1, min(20, p + random.choice([-1, 0, 1])))
        novas_posicoes.append(p)

    novo["posicao"] = novas_posicoes

    # resolve empates reordenando
    novo = novo.sort_values("posicao").reset_index(drop=True)
    novo["posicao"] = range(1, 21)

    return novo


# -----------------------------------------------------
# GERA 30 DIAS DE RANKING E INSERE NO BANCO
# -----------------------------------------------------
data_inicial = date(2025, 11, 1)
atual = df.copy()

for i in range(1, 30):  # já existe o dia 1
    dia = data_inicial + timedelta(days=i)

    atual = simular_variacao(atual)
    atual["data"] = dia

    with engine.begin() as conn:
        for _, row in atual.iterrows():
            stmt = text("""
                INSERT INTO ranking (id_filme, nota, posicao, data)
                VALUES (:id_filme, :nota, :posicao, :data)
            """)
            conn.execute(stmt, {
                "id_filme": int(row["id_filme"]),
                "nota": float(row["nota"]),
                "posicao": int(row["posicao"]),
                "data": dia
            })

print("Ranking de 30 dias inserido com sucesso.")


Ranking de 30 dias inserido com sucesso.
